In [50]:
import pandas as pd
import numpy as np
from itertools import combinations, product
from sklearn import tree

In [51]:
N_BINS = 10 #bin for continuos values 
N_VAR = 2 #max n. features to be modified
MAX = 5 #max counterfactuals


cats = ['marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country'] #categorical values
target = 'income' #target value


In [52]:
df = pd.read_json("adult_train.json")
Y = list(df[target])
Y = [bool(y) for y in Y]
X = df.loc[:, df.columns!=target]


In [53]:
binned_X = X.copy()
feats = dict()
for f in X.columns:
    if f in cats:
        feats[f] = X[f].unique()
    else:
        if len(X[f].unique()) <= N_BINS:
            feats[f] = X[f].unique()
        else:
            binned_X['bins'] = pd.cut(X[f], N_BINS)
            binned_X['median'] = binned_X.groupby('bins')[f].transform('median')
            feats[f] = binned_X['median'].unique()

In [54]:
model = tree.DecisionTreeClassifier()
model = model.fit(X, Y)

In [55]:
all_combinations = []
for i in range(N_VAR):
    combination = []
    for feat_comb in combinations(feats.keys(), i+1):
        combination.append(feat_comb)
    all_combinations.append(combination)

In [65]:
ok_feats = []
all_cf= []

for combination in all_combinations:
    for feat_set in combination:
        if len([f for f in feat_set if f in ok_feats]) == 0:
            list_of_values = []
            for f in feat_set:
                list_of_values.append(feats[f])
            cf_x = list(x.values[0])
            cf_x_model = x.copy()
            for val_comb in product(*list_of_values):
                if len([f for f in feat_set if f in ok_feats]) == 0 and len(all_cf) < MAX:
                    for val, f in zip(val_comb, feat_set):
                        idx = list(feats.keys()).index(f)
                        cf_x[idx] = val
                        cf_x_model.at[0,f]=val
                    if model.predict(cf_x_model) != model.predict(x) and cf_x_model.values.tolist()[0] != x.values.tolist()[0]:
                        all_cf.append(cf_x_model)
                        for f in feat_set:
                            ok_feats.append(f)
                    if len(all_cf) == MAX:
                        break